#### Optimize M2M100 model with ONNX


In this notebook we will describes steps to laod models with m2m100 translation models from HuggingFace and optimize them with ONNX Runtime. We will also show how to use the optimized model to perform translation.

Once the model are optimize we will deploy them as an Api so that they can be used in a web application.

At the first step we will load the vanilla model from Hugginface and use it for inference, then we will convert it to ONNX and Finally we will optimize it with ONNX Runtime.

### First Step

Loading the vanilla model from hugginface

In [1]:
from transformers import AutoTokenizer, M2M100ForConditionalGeneration, pipeline

/Users/es.py/Projects/Personal/masakhane-web/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_NAME = "masakhane/m2m100_418M_en_swa_rel_news"

In [3]:
model = M2M100ForConditionalGeneration.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [4]:
text_to_translate = "Hello, my name is Espoir Murhabazi,  I am a Software Engineer from Congo DRC but living in UK"

In [5]:
model_input = tokenizer(text_to_translate, return_tensors="pt")

In [6]:
generated_tokens = model.generate(**model_input, forced_bos_token_id=tokenizer.lang_code_to_id["sw"])

/Users/es.py/Projects/Personal/masakhane-web/.venv/lib/python3.10/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 200 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [7]:
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [8]:
translated_text

['Jina langu ni Espoir Murhabazi, Mimi ni mhandisi wa programu za kompyuta kutoka Kongo DRC lakini ninaishi Uingereza']

Trying to export the model manually and see if we can load the model.

In [9]:
MODEL_SUFFIX = MODEL_NAME.replace('masakhane/', '')

In [10]:
%%script false --no-raise-error
onnx_inputs, onnx_outputs = export_onnx(
    preprocessor=tokenizer,
    model=model,
    config=onnx_config,
    opset=13,
    output=output_path,
)

In [12]:
MODEL_SUFFIX

'm2m100_418M_en_swa_rel_news'

This command is not working properly, It is saving the model as one file instead of two file one for the encoder another one for the decoder.

The best approach is to use CLI as suggested in the documentation.

` optimum-cli export onnx --model masakhane/m2m100_418M_en_swa_rel_news --task seq2seq-lm-with-past --for-ort onnx`

In [13]:
! optimum-cli export onnx --model masakhane/m2m100_418M_en_swa_rel_news --task seq2seq-lm-with-past --for-ort onnx/m2m100_418M_en_swa_rel_news

Framework not specified. Using pt to export to ONNX.
Using framework PyTorch: 1.13.1
Overriding 1 configuration item(s)
	- use_cache -> False
/Users/es.py/Projects/Personal/masakhane-web/.venv/lib/python3.10/site-packages/transformers/models/m2m_100/modeling_m2m_100.py:172: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if max_pos > self.weights.size(0):
/Users/es.py/Projects/Personal/masakhane-web/.venv/lib/python3.10/site-packages/transformers/models/m2m_100/modeling_m2m_100.py:293: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() 

check if the model is correct

In [16]:
from pathlib import Path

In [17]:
base_model_onnx_dir = Path("onnx").joinpath(MODEL_SUFFIX)

In [18]:
base_model_onnx_dir

PosixPath('onnx/m2m100_418M_en_swa_rel_news')

### Use the optimization to Opimze the model

In this section we will apply the first optimization to the model we saved in the previous step.

We will start by testing the basica optimization to see 

In [23]:
from optimum.onnxruntime import ORTOptimizer
from optimum.onnxruntime.configuration import OptimizationConfig
from transformers import AutoConfig
from optimum.onnxruntime import ORTModelForSeq2SeqLM

In [22]:
optimization_config = OptimizationConfig(optimization_level=99)

/Users/es.py/Projects/Personal/masakhane-web/.venv/lib/python3.10/site-packages/optimum/onnxruntime/configuration.py:726: FutureWarning: disable_embed_layer_norm will be deprecated soon, use disable_embed_layer_norm_fusion instead, disable_embed_layer_norm_fusion is set to True.
  warnings.warn(


### Loading the Model

In [24]:
onnx_model =  ORTModelForSeq2SeqLM.from_pretrained(base_model_onnx_dir)

In [25]:
optimizer = ORTOptimizer.from_pretrained(onnx_model)

In [26]:
optimized_model_path = Path("onnx").joinpath(f"{MODEL_SUFFIX}_optimized/")
optimized_model_path.mkdir(parents=True, exist_ok=True)

In [27]:
optimizer.optimize(save_dir=optimized_model_path, optimization_config=optimization_config)

2023-03-01 15:37:47.100773 [W:onnxruntime:, inference_session.cc:1546 Initialize] Serializing optimized model with Graph Optimization level greater than ORT_ENABLE_EXTENDED and the NchwcTransformer enabled. The generated model may contain hardware specific optimizations, and should only be used in the same environment the model was optimized in.
symbolic shape infer failed. it's safe to ignore this message if there is no issue with optimized model
symbolic shape infer failed. it's safe to ignore this message if there is no issue with optimized model
symbolic shape infer failed. it's safe to ignore this message if there is no issue with optimized model
symbolic shape infer failed. it's safe to ignore this message if there is no issue with optimized model
symbolic shape infer failed. it's safe to ignore this message if there is no issue with optimized model
symbolic shape infer failed. it's safe to ignore this message if there is no issue with optimized model
symbolic shape infer failed.

PosixPath('onnx/m2m100_418M_en_swa_rel_news_optimized')

Using the optimize model and check if the model is working.

### Use the optimized model

Once we have developed the model, let us now use the optimized model to run the inference and check if the model is working.

In [28]:
from optimum.onnxruntime import ORTModelForSeq2SeqLM

In [29]:
optimized_model = ORTModelForSeq2SeqLM.from_pretrained(optimized_model_path)

In [30]:
from optimum.pipelines import pipeline

In [31]:
onnx_optimize = pipeline("translation_en_to_sw", model=optimized_model, tokenizer=tokenizer)

In [32]:
translated_text = onnx_optimize(text_to_translate)

In [33]:
translated_text

[{'translation_text': 'Jina langu ni Espoir Murhabazi, Mimi ni mhandisi wa programu za kompyuta kutoka Kongo DRC lakini ninaishi Uingereza'}]

I have managed to apply optimization and run the inference on the model, the last issue will be to run the test to check if the performance of the predicted model is good but at least the model is now working. I need to now move to the next step which is deploying the model.

### Applying Quantization

Learn more about quantization here..

In [34]:
from optimum.onnxruntime import ORTQuantizer, ORTModelForSeq2SeqLM
from optimum.onnxruntime.configuration import AutoQuantizationConfig

In [44]:
encoder_quantizer = ORTQuantizer.from_pretrained(base_model_onnx_dir, file_name="encoder_model.onnx")

In [45]:
decoder_quantizer = ORTQuantizer.from_pretrained(base_model_onnx_dir, file_name="decoder_model.onnx")

In [46]:
decoder_with_past_quantizer = ORTQuantizer.from_pretrained(base_model_onnx_dir, file_name="decoder_with_past_model.onnx")

In [47]:
quantizers = [encoder_quantizer, decoder_quantizer, decoder_with_past_quantizer]

In [48]:
dynamic_quantization_config = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

In [49]:
quantized_model_path = Path("onnx").joinpath(f"{MODEL_SUFFIX}_quantized/")
quantized_model_path.mkdir(parents=True, exist_ok=True)

In [50]:
for quantizer in quantizers:
    quantizer.quantize(quantization_config=dynamic_quantization_config, save_dir=quantized_model_path)

In [51]:
quantized_model = ORTModelForSeq2SeqLM.from_pretrained(quantized_model_path)

In [52]:
quantized_pipeline = pipeline("translation_en_to_sw", model=quantized_model, tokenizer=tokenizer)

In [53]:
translated_text_quantized = quantized_pipeline(text_to_translate)

In [54]:
print(translated_text_quantized)

[{'translation_text': 'Jina langu ni Espoir Murhabazi, Mimi ni mhandisi wa programu za kompyuta kutoka Kongo DRC lakini ninaishi Uingereza'}]


The quantization seems to reduce the size of the model but keeping the same performance, as per the documentaiton and experience performed on other models, we need to perform the quantization on other model to check for the performance.